In [1]:
from grace.simulate import random_graph
from grace.models.classifier import GCN
from grace.models.train import train_model
from grace.models.datasets import dataset_from_graph

import networkx as nx
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
G = random_graph()

In [ ]:
pos = {idx: (node["x"], node["y"]) for idx, node in G.nodes(data=True)}

In [ ]:
nx.draw_networkx(G, pos=pos, node_size=2, with_labels=True)

In [ ]:
dataset = []

RNG = np.random.default_rng()

for i in range(100):
    G = random_graph(
        n_lines=RNG.integers(0, 10),
        n_chaff=RNG.integers(50, 1000),
        )
    d = dataset_from_graph(G)

    dataset += d

In [ ]:
model = GCN(input_channels=3, hidden_channels=64)

train_model(model, dataset, epochs=100)

In [2]:
test_G = random_graph(n_lines=3, n_chaff=300)
test_dataset = dataset_from_graph(test_G)

KeyError: <GraphAttrs.NODE_GROUND_TRUTH: 'node_ground_truth'>

In [ ]:
from scipy.special import softmax

In [ ]:
model.eval()

predictions = []

for data in test_dataset:  # Iterate in batches over the training/test dataset.
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    probs = softmax(out.detach().numpy())

    predictions.append((int(pred.numpy()), probs))


In [ ]:
pos = {idx: (node["x"], node["y"]) for idx, node in test_G.nodes(data=True)}

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
nx.draw_networkx(test_G, ax=ax, pos=pos, node_size=2, with_labels=False)

x = [p[0] for _, p in pos.items()]
y = [p[1] for _, p in pos.items()]

c = [pc[:, 1] for p, pc in predictions] 

scx = ax.scatter(x, y, s=64, c=c, cmap=plt.cm.PiYG)
fig.colorbar(scx)

In [ ]:
len(predictions)

In [ ]:
torch.save(model, "./gcn.pt")

In [ ]:
import torch